In [5]:
# Run this whenever you shut down the VM
# !pip3 install cassandra-driver --user
# !pip install folium
# !pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 21.5MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
import datetime


import pandas as pd

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)
def connect():
    cluster = Cluster(
        contact_points=['172.31.22.32'], 
        auth_provider = PlainTextAuthProvider(username='tomcat', password='drawsweb'),
        connect_timeout = 20,
        control_connection_timeout=None
    )
    session = cluster.connect()
    session.set_keyspace('disruptionwarning')
    session.row_factory = pandas_factory
    session.default_fetch_size = 1000000 #needed for large queries, otherwise driver will do pagination. Default is 50000.
    return session

session=connect()

# Set variables for analysis

In [2]:
dest='KSFO'
airline='SKW'
flightno='5348'
fromdt=datetime.date(2019,11,4)
todt=datetime.date(2019,11,5)
regno='N213SY'

In [5]:
# Flights added to the registry from flightplans_destination table
# SELECT * FROM flightplans_destination where dest = 'KSFO' AND airline='UAL' and flightno='2744' and departure > '2019-10-13'  allow filtering;
query="SELECT * FROM airportarrivals where dest = ? AND airline=? and flightno=? and aon >= ? and aon <= ?  allow filtering"
fp_dest_stmt=session.prepare(query)
fp_dest_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

rows_dest = session.execute(fp_dest_stmt, [dest,airline,flightno,fromdt,todt])
df_dest = rows_dest._current_rows 
df_dest_part=df_dest[['dest','aon','airline','flightno']]
df_dest

# #Cancellation query
# query="SELECT * FROM cancellation WHERE airline=? AND flightno=? AND timestamp >= ? AND timestamp <= ?"
# fp_cancel_stmt=session.prepare(query)
# fp_cancel_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

# rows_cancel = session.execute(fp_cancel_stmt, [airline,flightno,fromdt,todt])
# df_cancel = rows_cancel._current_rows 
# df_cancel_part=df_cancel[['id','origin']]

# #joining based on id, in case origin is NaN then there are no records for the id in the cancellation table
# df_merge=pd.merge(df_dest_part,df_cancel_part,left_on='id',right_on='id', how='left')
# df_merge


,dest,aon,airline,flightno,ain,airlineiata,aoff,approach,approachalt,diversion,...,orig,origiata,runway,rwy,sarr,sdep,size,tail,timeinstar,wake
0,KSFO,2019-11-04 15:35:17,SKW,5348,2019-11-04 15:41:15,None,2019-11-04 13:14:21,DYAMD5D,27325,None,...,KABQ,None,28L/R,28R,2019-11-04 15:45:00,2019-11-04 13:04:00,0,N213SY,1214,0


In [4]:
#Scheduled arrival departure uid are updated based on schedules
query="SELECT * FROM schedules WHERE airline=? AND flightno=? AND departure >= ? and departure <= ? AND destination=? ALLOW FILTERING"
fp_dest_lookup_stmt=session.prepare(query)
fp_dest_lookup_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

rows = session.execute(fp_dest_lookup_stmt, [airline,flightno,fromdt,todt,dest])
df = rows._current_rows 
df


,airline,flightno,departure,arrival,destination,destterm,equipment,opairline,opflightno,origin,originterm


In [164]:
#Update on basis of tail and flightplans respectively            
query="SELECT * FROM flightplans_registration WHERE registration=? AND airline=? AND flightno=? AND timestamp>=? and timestamp <=?"

fp_regn_stmt=session.prepare(query)
fp_regn_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

rows_regn = session.execute(fp_regn_stmt, [regno,airline,flightno,fromdt,todt])
df_regn = rows_regn._current_rows 
df_regn_part=df_regn[['dest','departure','airline','flightno','received','id','registration','timestamp']]

df_regn_part

,dest,departure,airline,flightno,received,id,registration,timestamp
0,KSFO,2019-11-04 13:00:00,SKW,5348,2019-11-04 12:29:23,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
1,KSFO,2019-11-04 13:00:00,SKW,5348,2019-11-04 12:54:01,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
2,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 12:54:39,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
3,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 13:03:40,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
4,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 13:04:07,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
5,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 13:08:41,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
6,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 13:10:10,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
7,KSFO,2019-11-04 13:03:00,SKW,5348,2019-11-04 13:13:36,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
8,KSFO,2019-11-04 13:04:00,SKW,5348,2019-11-03 13:00:52,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00
9,KSFO,2019-11-04 13:04:00,SKW,5348,2019-11-03 13:05:46,SKW5348-1572673554-airline-0460,N213SY,2019-11-04 13:10:00


In [165]:
query="SELECT * FROM flightplans WHERE airline=? AND flightno=? AND departure>=? AND departure<=? AND dest=? ALLOW FILTERING"

fp_dest_lookup_stmt=session.prepare(query)
fp_dest_lookup_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

rows = session.execute(fp_dest_lookup_stmt, [airline,flightno,fromdt,todt,dest])
df = rows._current_rows 
df

,airline,flightno,departure,received,altitude,arrival,dest,duration,equipment,id,orig,registration,speed,status,timestamp,waypoints
0,SKW,5348,2019-11-04 13:00:00,2019-11-04 12:29:23,34000,2019-11-04 14:54:00,KSFO,114,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,453,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
1,SKW,5348,2019-11-04 13:00:00,2019-11-04 12:54:01,34000,2019-11-04 14:54:00,KSFO,114,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
2,SKW,5348,2019-11-04 13:03:00,2019-11-04 12:54:39,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
3,SKW,5348,2019-11-04 13:03:00,2019-11-04 13:03:40,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
4,SKW,5348,2019-11-04 13:03:00,2019-11-04 13:04:07,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
5,SKW,5348,2019-11-04 13:03:00,2019-11-04 13:08:41,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
6,SKW,5348,2019-11-04 13:03:00,2019-11-04 13:10:10,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
7,SKW,5348,2019-11-04 13:03:00,2019-11-04 13:13:36,34000,2019-11-04 15:01:00,KSFO,118,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,410,F,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
8,SKW,5348,2019-11-04 13:04:00,2019-11-03 13:00:52,36000,2019-11-04 15:08:12,KSFO,124,E170,SKW5348-1572673554-airline-0460,KABQ,N213SY,448,S,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."
9,SKW,5348,2019-11-04 13:04:00,2019-11-03 13:05:46,36000,2019-11-04 15:08:12,KSFO,124,E75L,SKW5348-1572673554-airline-0460,KABQ,N213SY,376,S,2019-11-04 13:10:00,"[{""lon"":-106.61,""lat"":35.04},{""lon"":-106.66,""l..."


In [166]:
#Cancellation query
query="SELECT * FROM cancellation WHERE airline=? AND flightno=? AND timestamp >= ? AND timestamp <= ?"
fp_dest_lookup_stmt=session.prepare(query)
fp_dest_lookup_stmt.consistency_level = ConsistencyLevel.LOCAL_ONE

rows = session.execute(fp_dest_lookup_stmt, [airline,flightno,fromdt,todt])
df = rows._current_rows 
df


,airline,flightno,timestamp,departure,destination,id,origin


In [141]:
pd.options.display.max_rows = 4000

import json
rawjson=df_dest[9:10]['waypoints'].astype(str)
aa=rawjson.to_string()

xx=aa.split('9    ')[1]
print(xx.to_string())
# parsed=json.loads(xx)

# print (json.dumps(parsed, indent=2, sort_keys=True))

AttributeError: 'str' object has no attribute 'to_string'

In [149]:
rawjson=df_dest[9:10]['waypoints'].apply(json.loads)
parsed=rawjson.to_json()
print(parsed)

{"9":[{"lon":-98.47,"lat":29.53},{"lon":-98.52,"lat":29.6},{"lon":-98.53,"lat":29.62},{"lon":-98.56,"lat":29.66},{"lon":-98.56,"lat":29.66},{"lon":-98.62,"lat":29.74},{"lon":-98.65,"lat":29.78},{"lon":-98.74,"lat":29.85},{"lon":-98.8,"lat":29.89},{"lon":-98.89,"lat":29.95},{"lon":-98.9,"lat":29.95},{"lon":-98.94,"lat":29.98},{"lon":-98.98,"lat":30.01},{"lon":-99.12,"lat":30.11},{"lon":-99.21,"lat":30.17},{"lon":-99.35,"lat":30.26},{"lon":-99.37,"lat":30.27},{"lon":-99.37,"lat":30.27},{"lon":-99.53,"lat":30.3},{"lon":-99.61,"lat":30.32},{"lon":-99.82,"lat":30.36},{"lon":-99.89,"lat":30.37},{"lon":-100.03,"lat":30.4},{"lon":-100.25,"lat":30.44},{"lon":-100.26,"lat":30.45},{"lon":-101.03,"lat":30.6},{"lon":-101.42,"lat":30.67},{"lon":-102.27,"lat":30.83},{"lon":-102.98,"lat":30.95},{"lon":-103.45,"lat":31.08},{"lon":-105.38,"lat":31.59},{"lon":-106.23,"lat":31.8},{"lon":-106.28,"lat":31.82},{"lon":-108.2,"lat":32.39},{"lon":-108.9,"lat":32.59},{"lon":-110,"lat":33},{"lon":-110.5,"lat":33.